In [12]:
import numpy as np
import heapq
import random
from Model import Bin
from Model import Objet
import sys
import os
cwd = os.getcwd()
print(cwd)
from Méta_Heuristiques.Recuit_Sim2 import RS
from Instances_reader import ReadInstance

/home/nsarah/Documents/2CS-SIQ3-S2/2019-2020/OPT/TP/GitHub/BPP_Solver


In [13]:
def makeObjects(filepath):
    liste = []
    file1 = open(filepath, 'r')
    n = int(file1.readline())
    c = int(file1.readline())
    for i in range(n):
        obj = file1.readline()
        liste.append(int(obj))
    listObjects = []
    for i in range(len(liste)):
        listObjects.append(Objet(i+1,liste[i]))
    return n,c,listObjects

In [15]:
crossOverProb = 85  # 0.85 cross-over probability, 85% chance
mutationProb = 10  # 0.1 mutation probability, 10% chance


def Chromosome(object):
    def __init__(self, liste):  # liste des gènes
        self._chromosome = liste[1:]
        self._n = liste[0]  # nombre de boites

    @property
    def chromosome(self):  # Retourne le nom de l'objet.
        return self._chromosome

    @property
    def solution(self):  # retourne la liste des boites et leurs objets
        liste = self._chromosome
        cell0 = liste[0]
        n = len(liste) - 1  # la longueur du chromosome sans la cellule 0
        resultat = []
        for j in range(2 * n):
            resultat.append([])
        liste = copy.deepcopy(liste[1:])  # elimination de la cellule0
        lastB = 1  # compte le nombre de boites utilisées
        for i in range(n):
            c = liste[i]  # numero du gène
            a = (c + 1) % n
            b = int((c + 1) / n)
            if a == 0:
                # print("objet{}, boite{}".format(i+1,b))
                resultat[b].append(i + 1)
                if b > lastB:
                    lastB = b
            else:
                resultat[b + 1].append(i + 1)
                if b > lastB:
                    lastB = b
            # print("objet{}, boite{}".format(i+1,b+1))
        liste1 = [x for x in resultat if x != []]  # elimination es boites vides

        return liste1, len(
            liste1)  # lastb+1 = nombre de boites utilisée #la listee : liste des boites avec leurs objets dedans


def renameItems(liste):
    listeItems = []
    for i in range(len(liste)):
        item = Objet(int(i + 1), int(liste[i]))
        listeItems.append(item)
    return listeItems


def first_fit(list_items, max_size):
    """ Returns list of bins with input items inside. """
    list_bins = []
    list_bins.append(Bin(1, max_size))  # Add first empty bin to list

    cptBins = 1
    for item in list_items:
        # Go through bins and try to allocate
        alloc_flag = False

        for bin in list_bins:
            if bin.total_weight + item.weight <= max_size:
                bin.ranger_obj(item)
                alloc_flag = True
                break

        # If item not allocated in bins in list, create new bin
        # and allocate it to it.
        if alloc_flag == False:
            cptBins += 1
            newBin = Bin(cptBins, max_size)
            newBin.ranger_obj(item)
            list_bins.append(newBin)

    # Turn bins into list of items and return
    list_items = []
    for bin in list_bins:
        list_items.append(bin.get_objects)

    return (list_items), len(list_items)  # nombre de boites


def itemsInBox(liste):
    # donne l'emplacement de chaque objet
    boxes = []
    items = []
    for i in range(len(liste)):
        # items =[]
        for j in range(len(liste[i])):
            string = str(liste[i][j])[1:-2]
            number = string.split(',')[0]
            weight = string.split(',')[1]
            # thing = str(str(i)+','+number+','+str(j))
            thing = str(number + ',' + str(i + 1) + ',' + weight + ',' + str(
                j))  # thing = num_objet,num_boite,weight, pos_boite
            items.append(thing)
        # boxes.append(items)
    return items  # liste des objet sformatés selon leurs boites et leurs positions dans les boites


def toChromosome(liste,
                 n):  # liste dont chaque index= num boite, et liste[index] contient les nums des objets contenus dans la boite
    # n : nombre de boites utilisée
    chromosome = []
    chromosome.append(n)
    for i in range(1, len(liste) + 1):
        chromosome.append(0)
    number = len(liste)  # nombre d'objets
    lastIndex = 1
    lastBoite = 1
    for i in range(len(liste)):  # chaque objet
        string = str(liste[i])
        item = int(string.split(',')[0])
        boite = int(string.split(',')[1])
        weight = int(string.split(',')[2])
        pos = int(string.split(',')[3])
        if int(boite) == lastBoite:
            chromosome[int(item)] = lastIndex + int(pos)
            lastIndex = lastIndex + pos
            lastBoite = int(boite)
        else:
            chromosome[int(item)] = lastIndex + number
            lastIndex = lastIndex + number
            lastBoite = int(boite)

    return chromosome


def chromoTo(liste, n):
    chrom = []
    for i in range(len(liste)):
        chrom.append([])
    for i in range(len(liste)):  # chaque objet
        string = str(liste[i])
        item = int(string.split(',')[0])
        boite = int(string.split(',')[1])
        weight = int(string.split(',')[2])
        pos = int(string.split(',')[3])
        chrom[item - 1] = (boite - 1) * len(liste) + pos
    ch = []
    ch.append(n)
    for i in chrom:
        ch.append(i)
    return ch


def solution(liste):  # retourne la liste des boites et leurs objets
    # la liste en entrée est tout le chromosome avec la cellule zero
    cell0 = liste[0]
    n = len(liste) - 1  # la longueur du chromosome sans la cellule 0
    resultat = []
    for j in range(2 * n):
        resultat.append([])
    liste = copy.deepcopy(liste[1:])  # elimination de la cellule0
    lastB = 1  # compte le nombre de boites utilisées
    for i in range(n):
        c = liste[i]  # numero du gène
        a = (c + 1) % n
        b = int((c + 1) / n)
        if a == 0:
            # print("objet{}, boite{}".format(i+1,b))
            resultat[b].append(i + 1)
            if b > lastB:
                lastB = b
        else:
            resultat[b + 1].append(i + 1)
            if b > lastB:
                lastB = b
            # print("objet{}, boite{}".format(i+1,b+1))
    liste1 = [x for x in resultat if x != []]  # elimination es boites vides

    return liste1, len(
        liste1)  # lastb+1 = nombre de boites utilisée #la listee : liste des boites avec leurs objets dedans


import copy


def generation_ff(listeItems, c, popSize):
    population = []
    longueur = len(listeItems)
    listeItems1 = copy.deepcopy(listeItems)
    # popSize=int((2**(longueur))/(2**(longueur-5)))
    for i in range(popSize):
        liste1 = renameItems(listeItems1)
        liste2, n = first_fit(liste1, c)

        # ------représenter la solution par la représentation chromosomique-------
        liste3 = itemsInBox((liste2))
        res = chromoTo(liste3, n)
        # --------------------------------------------------
        population.append(res)
        random.shuffle(listeItems1)
    return population


# remarque le chromosome généré par ff ne contient pas la cellule zero

def selection(k, popSize, population):
    # Selection of parents randomly by K-Way tournament
    parents = []
    while (len(parents) != popSize):
        index = int(random.random() * (len(population) - 1))
        fittestChromosome = population[index]
        for i in range(k):
            index = int(random.random() * (len(population) - 1))
            if (fittestChromosome[0] > population[index][0]):
                fittestChromosome = copy.deepcopy(population[index])
        parents.append(fittestChromosome)
    return parents


def scramble(chromosome):
    liste = copy.deepcopy(chromosome[1:])
    n = len(liste)
    listeIndices = []
    for i in range(n):
        listeIndices.append(i)
    nombreDeGenes = random.randrange(1, int(n / 2))  # le nombre de genes dans le sous-ensemble à permuter
    indicesPermutation = random.sample(listeIndices, nombreDeGenes)  # le sous ensemble des indices des gènes à permuter
    if len(indicesPermutation) != 1:
        indicesPermutés = indicesPermutation.copy()
        random.shuffle(indicesPermutés)  # la liste des indices permutés
        for i in range(n):
            for j in range(len(indicesPermutation)):
                if i == indicesPermutation[j]:
                    liste[i] = liste[indicesPermutés[j]]
        liste.insert(0, 0)
        liste2, nombreDeBoites = solution(liste)
        liste[0] = nombreDeBoites
    else:
        liste = copy.deepcopy(chromosome)
    return liste


def crossOverMutationRate(prob):
    chance = random.randint(0, 100)
    if (prob >= chance):
        return True
    else:
        return False


# Returns a random cross-over point
def getCrossOverPoint(taille):
    point = 0
    while (point == 0):
        point = random.randint(1, taille)
    return point


def crossover_mutation(population, taille, nbElt, listeWeight, c):
    random.shuffle(population)
    newpopulation = []
    index = 1
    while ((index <= len(population)) and (len(newpopulation) != taille)):
        if (index < len(population)):
            pchrom1 = copy.deepcopy(population[index - 1])
            pchrom2 = copy.deepcopy(population[index])
            fchrom1 = []
            fchrom2 = []
            if (crossOverMutationRate(crossOverProb)):
                point = getCrossOverPoint(len(pchrom1))
                for i in range(len(pchrom1)):
                    if (i == 0):
                        fchrom1.append(pchrom1[i])
                        fchrom2.append(pchrom2[i])
                    if (i != 0):
                        if (i <= point):  # Before and up to cross-over point...
                            fchrom1.append(pchrom1[i])
                            fchrom2.append(pchrom2[i])
                        else:  # After cross-over point...
                            fchrom1.append(pchrom2[i])
                            fchrom2.append(pchrom1[i])
                fchrom11, n1 = solution(fchrom1)
                fchrom22, n2 = solution(fchrom2)
                fchrom1[0] = n1
                fchrom2[0] = n2
            else:
                # Else no cross-over
                fchrom1 = copy.deepcopy(pchrom1)
                fchrom2 = copy.deepcopy(pchrom2)
            if (crossOverMutationRate(mutationProb)):
                fchrom111 = scramble(fchrom1)
                fchrom222 = scramble(fchrom2)
            else:
                fchrom111 = copy.deepcopy(fchrom1)
                fchrom222 = copy.deepcopy(fchrom1)
            sol1, n11 = solution(fchrom111)
            stop1 = fitness(sol1, nbElt, listeWeight, c)
            sol2, n22 = solution(fchrom222)
            stop2 = fitness(sol2, nbElt, listeWeight, c)
            if not stop1:
                fchrom111 = copy.deepcopy(pchrom1)
            elif not stop2:
                fchrom222 = copy.deepcopy(pchrom2)
            newpopulation.append(fchrom111)
            newpopulation.append(fchrom222)
        else:
            pchrom1 = copy.deepcopy(population[index - 1])
            fchrom1 = copy.deepcopy(pchrom1)
            if (crossOverMutationRate(mutationProb)):
                fchrom111 = scramble(fchrom1)
            else:
                fchrom111 = copy.deepcopy(fchrom1)
            sol1, n1 = solution(fchrom111)
            stop1 = fitness(sol1, nbElt, listeWeight, c)
            if not stop1:
                fchrom111 = copy.deepcopy(pchrom1)
            newpopulation.append(fchrom111)
        index = index + 2
    return newpopulation


def getCell0(chromosome):
    return chromosome[0]


def fitness(liste, nbElt, listeWeight, c):
    i = 0
    for p in liste:
        i = i + len(p)
    if (i < nbElt):
        return False
    else:
        for o in range(len(liste)):
            w = 0
            for k in range(len(liste[o])):
                w = w + listeWeight[liste[o][k] - 1]
            if w > c:
                return False
        return True


def evaluation(population, listeWeight, c, nb_elt):
    found = False
    j = 0
    listeX = []
    log = []
    listeM = []
    for i in population:
        listeX.append(getCell0(i))
        res, n = solution(i)
        log.append(fitness(res, nb_elt, listeWeight, c))
        j += 1
    for g in range(len(log)):
        if log[g] == True:
            listeM.append(listeX[g])
    indexes = []
    if True in log:
        for k in range(len(listeX)):
            if log[k] == True:
                if listeX[k] == min(listeM):
                    indexes.append(k)
        retour = []
        for a in indexes:
            retour.append(population[a])
        if (len(retour) > 0):
            found = True
        else:
            found = False
            retour = []
    else:
        found = False
        retour = []
    return retour, found


def next_generation(population, popSize, k, n, c, liste):  # le k de selection
    parents = selection(k, popSize, population)
    newpopulation = crossover_mutation(parents, popSize, n, liste, c)
    populationMuted = []
    for i in newpopulation :
        populationMuted.append(solution(i))
    RSresult=[]
    for j in populationMuted :
        copie=[]
        copie=j[0]
        binList=[]
        last=0
        for i in range(len(copie)) :
            boite =Bin(i+1,c)
            for k in range(len(copie[i])):
                thingy=copie[i]
                objecte = Objet(thingy[k],liste[thingy[k]-1])
                boite.ranger_obj(objecte)
            last+=k
            binList.append(boite)
        classrs = RS()
        rs, listeBins=classrs.RS_iteratif(n, c, j[0], S=binList, Tinit=30, T0=0.1, R=1000, alpha=0.925)
        solutionlist=[]
        for a in listeBins:
            solutionlist.append(a.get_objects)
        final =itemsInBox(solutionlist)
        RSresult.append(chromoTo(final, rs))
    return newpopulation #RSresult  


def main(nbGen, k, popSize, n, c, liste):  # le k de selection
    # n,c,liste =ReadInstance("N1C1W1_D.txt")
    population = generation_ff(liste, c, popSize)
    bestfit, found = evaluation(population, liste, c, n)
    if found:
        for i in range(nbGen):
            nx_gen = next_generation(population, popSize, k, n, c, liste)
            liste1, found1 = evaluation(nx_gen, liste, c, n)
            if found1:
                if (bestfit[0] > liste1[0]):
                    bestfit = copy.deepcopy(liste1)
            population = copy.deepcopy(nx_gen)
    return bestfit


def countFreq(arr, n):
    visited = [False for i in range(n)]
    indices = []
    counts = []
    for i in range(n):
        if (visited[i] == True):
            continue

        count = 1
        for j in range(i + 1, n, 1):
            if (arr[i] == arr[j]):
                visited[j] = True
                count += 1
        indices.append(i)
        counts.append(count)
    index_max = [i for i, x in enumerate(counts) if x == max(counts)]
    return arr.index(arr[index_max[0]]), counts[index_max[0]]


# retourne l'indice du premier max cell0 et le max cell0
n = 5
c = 10
list = [1, 1, 9, 4, 9, 7, 3, 2, 2, 9]
#n,c,liste = ReadInstance("C:\\Users\\winsido\\Documents\\GitHub\\BPP_Solver\\Instances_scholl\\classe1\\N1C1W1_D.txt")
sol = main(3, 2, 10, n, c, list)
sol1, n1 = solution(sol[0])
print(n1)


6


In [28]:
import os
import time
def ReadInstance(filepath):
    list = []
    file1 = open(filepath, 'r')
    n = int(file1.readline())
    c = int(file1.readline())
    for i in range(n):
        obj = file1.readline()
        list.append(int(obj))
    return n, c, list
def TestMatrix_class1():
    indications_class1 = ["N1C1", "N1C2", "N1C3", "N2C1", "N2C2", "N2C3", "N3C1", "N3C2", "N3C3", "N4C1", "N4C2",
                          "N4C3"]
    sol_moy=0
    t_exec=0
    cpt=0
    sum=0
    for indication in indications_class1:
        print("______CLASSE 01 ___________")
        directory = "/home/nsarah/Documents/2CS-SIQ3-S2/2019-2020/OPT/TP/GitHub/BPP_Solver/Instances_scholl/classe1"
        #directory ="C:/Users/winsido/Documents/GitHub/BPP_Solver/Instances_scholl/classe1"
        for filename in os.listdir(directory):
            if filename.endswith(".txt") and filename.startswith(indication):
                n, c, liste = ReadInstance(directory + "/" + filename)
                vectSolution =[]
                vectCell0 =[]
                tab_exec=[]
                iteration = 10
                for z in range(iteration) :
                    start_time = time.time()
                    sol = main(1,20,10,n,c,liste)
                    t_exec = time.time() - start_time
                    vectSolution.append(sol)
                    tab_exec.append(t_exec)
                    if(len(sol)>0):
                        vectCell0.append(sol[0][0])
                index=vectCell0.index(min(vectCell0))
                sol = vectSolution[index]
                print(filename,sol[0][0])
                sol1,n1=solution(sol[0])
                print(filename,sol1,n1)
                print(fitness(sol1,n,liste,c))
                if tab_exec[index] != -1:
                    sum = sum + tab_exec[index]
                    cpt = cpt + 1
                file = open("AG-RS_Classe1_1_k20_size10.txt", "a")
                item=filename.split(".")
                file.write(item[0]+"\t"+str(sol[0][0])+"\t"+str(tab_exec[index])+"\n")
                file.close()
        
        file = open("AG-RS_Classe1_1_k20_size10_moy.txt", "a")
        file.write(str(indication)+"\t"+str(sum/cpt)+"\n")
        file.close()

def TestMatrix_class2():
    indications_class2 = ["N1","N2", "N3", "N4"]
    sol_moy=0
    t_exec=0
    sum=0
    cpt=0
    for indication in indications_class2:
        print("______CLASSE 02 ___________")
        directory = "/home/nsarah/Documents/2CS-SIQ3-S2/2019-2020/OPT/TP/GitHub/BPP_Solver/Instances_scholl/classe2"
        #directory ="C:/Users/winsido/Documents/GitHub/BPP_Solver/Instances_scholl/classe2"
        for filename in os.listdir(directory):
            if filename.endswith(".txt") and filename.startswith(indication):
                n, c, liste = ReadInstance(directory + "/" + filename)
                vectSolution =[]
                vectCell0 =[]
                tab_exec=[]
                iteration = 10
                for z in range(iteration) :
                    start_time = time.time()
                    sol = main(1,20,10,n,c,liste)
                    t_exec = time.time() - start_time
                    vectSolution.append(sol)
                    tab_exec.append(t_exec)
                    if(len(sol)>0):
                        vectCell0.append(sol[0][0])
                index=vectCell0.index(min(vectCell0))
                sol = vectSolution[index]
                print(filename,sol[0][0])
                sol1,n1=solution(sol[0])
                print(filename,sol1,n1)
                print(fitness(sol1,n,liste,c))
                if tab_exec[index] != -1:
                    sum = sum + tab_exec[index]
                    cpt = cpt + 1
                file = open("AG-RS_Classe2_1_k20_size10.txt", "a")
                item=filename.split(".")
                file.write(item[0]+"\t"+str(sol[0][0])+"\t"+str(t_exec)+"\n")
                file.close()
        file = open("AG-RS_Classe2_1_k20_size10_moy.txt", "a")
        file.write(str(indications_class2)+"\t"+str(sum/cpt)+"\n")
        file.close()
                
def TestMatrix_class3():
    print("______CLASSE 03: (C=100000) ___________")
    sum=0
    cpt=0
    directory = "/home/nsarah/Documents/2CS-SIQ3-S2/2019-2020/OPT/TP/GitHub/BPP_Solver/Instances_scholl/classe3"
    #directory ="C:/Users/winsido/Documents/GitHub/BPP_Solver/Instances_scholl/classe3"
    for filename in os.listdir(directory):
            if filename.endswith(".txt") and filename.startswith("HARD"):
                n, c, liste = ReadInstance(directory + "/" + filename)
                vectSolution =[]
                vectCell0 =[]
                tab_exec=[]
                iteration = 10
                for z in range(iteration) :
                    start_time = time.time()
                    sol = main(1,20,10,n,c,liste)
                    t_exec = time.time() - start_time
                    vectSolution.append(sol)
                    tab_exec.append(t_exec)
                    if(len(sol)>0):
                        vectCell0.append(sol[0][0])
                index=vectCell0.index(min(vectCell0))
                sol = vectSolution[index]
                print(filename,sol[0][0])
                sol1,n1=solution(sol[0])
                print(filename,sol1,n1)
                print(fitness(sol1,n,liste,c))
                if tab_exec[index] != -1:
                    sum = sum + tab_exec[index]
                    cpt = cpt + 1
                file = open("AG-RS_Classe3_1_k20_size10.txt", "a")
                item=filename.split(".")
                file.write(item[0]+"\t"+str(sol[0][0])+"\t"+str(t_exec)+"\n")
                file.close()
    print(sum/cpt)
            



In [23]:
TestMatrix_class3()

______CLASSE 03: (C=100000) ___________


KeyboardInterrupt: 

In [29]:
TestMatrix_class1()

______CLASSE 01 ___________
N1C1W2_K.txt 35
N1C1W2_K.txt [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14, 48], [15, 49], [16, 44], [17, 43], [18, 45], [19, 42], [20, 46], [21, 47], [22, 50], [23], [24, 40], [25, 41], [26], [27], [28], [29], [30], [31, 34], [32, 33], [35, 36], [37, 38], [39]] 35
True
N1C1W4_D.txt 38
N1C1W4_D.txt [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23, 50], [24], [25], [26, 48], [27, 49], [28], [29, 43], [30, 41], [31, 40], [32, 42], [33, 39], [34, 44], [35, 45], [36, 38], [37, 46], [47]] 38
True
N1C1W1_D.txt 28
N1C1W1_D.txt [[1], [2], [3, 49], [4, 48], [5, 50], [6, 47], [7, 45], [8, 46], [9], [10, 40], [11, 41], [12, 36], [13, 33], [14, 32], [15, 34], [16, 35], [17, 37], [18, 38], [19, 31], [20, 39, 42], [21, 43, 44], [22], [23], [24], [25, 30], [26], [27, 28], [29]] 28
True
N1C1W1_F.txt 27
N1C1W1_F.txt [[1], [2, 49], [3, 50], [4], [5, 48], [6, 46], [7, 47]

N2C2W4_B.txt 60
N2C2W4_B.txt [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15, 100], [16], [17], [18], [19, 97], [20, 93], [21, 94], [22, 95], [23, 96], [24, 83], [25, 84], [26, 81], [27, 82], [28, 85], [29, 80], [30, 76], [31, 77], [32, 78], [33, 79], [34, 75], [35, 86], [36, 87], [37, 73], [38, 74], [39, 88], [40, 89], [41, 71], [42, 72], [43, 90], [44, 70], [45, 91], [46, 92], [47, 69], [48, 68], [49, 98], [50, 99], [51], [52, 67], [53], [54, 65], [55, 60], [56, 59], [57, 58], [61, 62], [63, 64], [66]] 60
True
N2C2W2_J.txt 56
N2C2W2_J.txt [[1, 99], [2, 97], [3, 98], [4, 100], [5], [6, 96], [7, 95], [8, 94], [9], [10], [11], [12], [13, 92], [14, 91], [15, 93], [16], [17], [18, 89], [19, 88], [20, 86], [21, 87], [22, 85], [23, 83], [24, 84], [25, 90], [26], [27, 79], [28, 80], [29, 81], [30, 77], [31, 78], [32, 82], [33], [34], [35, 74], [36, 73], [37, 75], [38, 72], [39, 76], [40], [41, 70], [42, 65], [43, 64], [44, 61], [45, 60], [46, 59], [47, 56], [4

KeyboardInterrupt: 

In [ ]:
TestMatrix_class2()

In [ ]:
TestMatrix_class3()